# Purpose:
- Develop "whisker GLM"
- Similar to "touch GLM" (240131_glm_test.ipynb)
- whisker feature values outside of touch frames were set to 0, AFTER standardization.
    - This works because the model is about "variation" in neuronal activity upon touch, and whisker features are going to be standardized.
    - Outside of touch frames will have no effect from the coefficients of whisker features, and bias will take care of negative-positive response distribution upon whisker features.


In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from pathlib import Path
import matplotlib.pyplot as plt

base_dir = Path(r'E:\TPM\JK\h5')
results_dir = Path(r'E:\TPM\JK\results')

expert_mice_df = pd.read_csv(base_dir / 'expert_mice.csv', index_col=0)
use_mice_df = expert_mice_df.loc[expert_mice_df['depth_matched'].astype(bool) & 
                                 ~expert_mice_df['processing_error'].astype(bool) &
                                 ((expert_mice_df.session_type == 'training') |
                                  (expert_mice_df.session_type.str.contains('test')))]

In [2]:
def standardize_with_nan_values(data):
    mean = np.nanmean(data)
    std = np.nanstd(data)
    standardized_data = (data - mean) / std
    nan_inds = np.where(np.isnan(standardized_data))[0]
    standardized_data[nan_inds] = np.nanmean(standardized_data)
    return standardized_data


def make_design_dataframe(mouse, plane, session, base_dir,
                          whisker_feature_offsets=np.arange(0,3),
                          whisking_offsets=np.arange(-2,5),
                          lick_offsets=np.arange(-2,3),
                          sound_offsets=np.arange(0,5),
                          reward_offsets=np.arange(0,5)):
    plane_dir = base_dir / f'{mouse:03}/plane_{plane}'
    behavior_fn = plane_dir / f'JK{mouse:03}_S{session:02}_plane{plane}_frame_whisker_behavior.pkl'
    if not behavior_fn.exists():
        raise FileNotFoundError(f'{behavior_fn} does not exist')
    behavior_frametime = pd.read_pickle(plane_dir / f'JK{mouse:03}_S{session:02}_plane{plane}_frame_whisker_behavior.pkl')
    roi_dir = plane_dir / f'{session:03}/plane0/roi'
    ophys_fn = roi_dir / 'refined_frame_time.pkl'
    if not ophys_fn.exists():
        raise FileNotFoundError(f'{ophys_fn} does not exist')
    ophys_frametime = pd.read_pickle(roi_dir / 'refined_frame_time.pkl')

    # remove those with remove_trial==True (from expert_mice.csv)
    refined_ophys_frametime = ophys_frametime.query('remove_trial==False')
    assert refined_ophys_frametime.remove_frame.values.sum() == 0
    # extend each trial frames by 1 in each direction (those trimmed to make reduced_frame_time.pkl from frame_time.pkl)
    # so that I can have 2 more frame of information (from behavior_frametime)
    extended_ophys_df = refined_ophys_frametime.groupby('trialNum').apply(extend_dataframe).reset_index(drop=True)

    # merge with behavior_frametime
    reduced_behavior_columns = np.setdiff1d(behavior_frametime.columns,
                                            np.setdiff1d(extended_ophys_df.columns,
                                                         ['trialNum', 'frame_index']))
    reduced_behavior_df = behavior_frametime[reduced_behavior_columns]
    merged_df = pd.merge(extended_ophys_df, reduced_behavior_df,
                         on=['trialNum', 'frame_index'], how='inner')

    # remove catch trials
    catch_trial_nums = merged_df.query('trial_type == "oo"')['trialNum'].unique()
    merged_df = merged_df.query('trialNum not in @catch_trial_nums').reset_index().copy()
    assert 'oo' not in merged_df['trial_type'].unique()
    
    # Assign pole in sound and pole out sound cue frames
    assert not merged_df.groupby('trialNum').apply(lambda x: len(np.where(x['pole_up_frame']==1)[0])==0).any()
    merged_df['pole_in_frame'] = merged_df.groupby('trialNum').apply(lambda x: x['frame_index'] == x['frame_index'].values[np.where(x['pole_up_frame']==True)[0][0]-1]).reset_index(drop=True).values    
    merged_df['pole_out_frame'] = merged_df.groupby('trialNum').apply(apply_pole_out).reset_index(drop=True).values

    # Initialize names
    whisker_feature_names = ['theta_onset', 'phi_onset', 'kappaH_onset', 'kappaV_onset',
                            'arc_length_onset', 'touch_count', 'delta_theta', 'delta_phi',
                            'delta_kappaH', 'delta_kappaV', 'touch_duration', 'slide_distance']
    lick_names = ['num_lick_left', 'num_lick_right']
    whisking_names = ['num_whisks', 'midpoint', 'amplitude']
    reward_names = ['first_reward_lick_left', 'first_reward_lick_right']
    sound_names = ['pole_in_frame', 'pole_out_frame']

    # Standardize and apply mean after standardization for whisker features
    # except for touch count (replace nan to 0 for touch count)
    for whisker_feature_name in whisker_feature_names:
        if whisker_feature_name != 'touch_count':
            merged_df.loc[:,whisker_feature_name] = standardize_with_nan_values(merged_df.loc[:,whisker_feature_name].values)
    merged_df[f'touch_count'] = merged_df[f'touch_count'].apply(lambda x: 0 if np.isnan(x) else x)

    # Build design dataframe
    design_df = merged_df[['trialNum','frame_index']].copy()
    for whisker_feature_name in whisker_feature_names:
        for offset in whisker_feature_offsets:
            values_to_assign = merged_df.groupby('trialNum').apply(lambda x: x[whisker_feature_name].shift(offset)).reset_index(drop=True).values
            design_df.loc[:,f'{whisker_feature_name}_{offset}'] = values_to_assign
    for whisking_name in whisking_names:
        for offset in whisking_offsets:
            design_df[f'{whisking_name}_{offset}'] = merged_df.groupby('trialNum').apply(lambda x: x[whisking_name].shift(offset)).reset_index(drop=True).values
    for lick_name in lick_names:
        for offset in lick_offsets:
            design_df[f'{lick_name}_{offset}'] = merged_df.groupby('trialNum').apply(lambda x: x[lick_name].shift(offset)).reset_index(drop=True).values
    for sound_name in sound_names:
        for offset in sound_offsets:
            design_df[f'{sound_name}_{offset}'] = merged_df.groupby('trialNum').apply(lambda x: x[sound_name].shift(offset)).reset_index(drop=True).values
    for reward_name in reward_names:
        for offset in reward_offsets:
            design_df[f'{reward_name}_{offset}'] = merged_df.groupby('trialNum').apply(lambda x: x[reward_name].shift(offset)).reset_index(drop=True).values

    return design_df, merged_df


def extend_dataframe(group, n_before=1, n_after=1):
    before_rows = group.iloc[0:n_before].copy().reset_index(drop=True)
    before_rows[:] = np.nan
    before_rows.trialNum = group.trialNum.iloc[0]
    before_rows.frame_index = -1
    before_rows.loc[before_rows.index.max(), 'frame_index'] = group.frame_index.min()-1
    after_rows = group.iloc[-n_after:].copy().reset_index(drop=True)
    after_rows[:] = np.nan
    after_rows.trialNum = group.trialNum.iloc[-1]
    after_rows.frame_index = -1
    after_rows.loc[0,'frame_index'] = group.frame_index.max()+1
    extended_group = pd.concat([before_rows, group, after_rows], ignore_index=True)
    return extended_group


def apply_pole_out(x):
     if np.where(x['pole_up_frame']==True)[0][-1] < len(x)-1:
          return x['frame_index'] == x['frame_index'].values[np.where(x['pole_up_frame']==True)[0][-1]]
     else:
          return pd.Series([False]*len(x))

# Design matrix dev for whisker GLM
- Use touch whisker features 
    - Calculated and saved by ..\data_processing\240328_build_whisker_feature_dataframe_dev.ipynb
- Other features the same as from touch GLM, except for pole angle

In [7]:
mouse = 25
plane = 1
session = 3
plane_dir = base_dir / f'{mouse:03}/plane_{plane}'
behavior_frametime = pd.read_pickle(plane_dir / f'JK{mouse:03}_S{session:02}_plane{plane}_frame_whisker_behavior.pkl')
roi_dir = plane_dir / f'{session:03}/plane0/roi'
ophys_frametime = pd.read_pickle(roi_dir / 'refined_frame_time.pkl')
refined_ophys_frametime = ophys_frametime.query('remove_trial==False').reset_index()
assert refined_ophys_frametime.remove_frame.values.sum() == 0
extended_ophys_df = refined_ophys_frametime.groupby('trialNum').apply(extend_dataframe).reset_index(drop=True)
reduced_behavior_columns = np.setdiff1d(behavior_frametime.columns,np.setdiff1d(extended_ophys_df.columns, ['trialNum', 'frame_index']))
reduced_behavior_df = behavior_frametime[reduced_behavior_columns]
merged_df = pd.merge(extended_ophys_df, reduced_behavior_df, on=['trialNum', 'frame_index'], how='outer')
    # Here outer join is to keep behavior data preceeding and following ophys frames
constant_columns = ['correct', 'wrong', 'miss', 'trial_type', 'task_target', 'distractor', 'mouse_name', 'session_name', 'session_type', 'trial_duration']
# Assigne pole_moving_up and pole_moving_down to the frames
# First check if all trials have correct pole up pole moving frames
# Sometimes there is no pole_moving_frame
# Just use -1 of the first pole up and the last of pole up frame as pole_in_frame and pole_out_frame
assert not merged_df.groupby('trialNum').apply(lambda x: len(np.where(x['pole_up_frame']==1)[0])==0).any()
merged_df['pole_in_frame'] = merged_df.groupby('trialNum').apply(lambda x: x['frame_index'] == x['frame_index'].values[np.where(x['pole_up_frame']==True)[0][0]-1]).reset_index(drop=True).values
merged_df['pole_out_frame'] = merged_df.groupby('trialNum').apply(lambda x: x['frame_index'] == x['frame_index'].values[np.where(x['pole_up_frame']==True)[0][-1]]).reset_index(drop=True).values



In [11]:
whisker_feature_names = ['theta_onset', 'phi_onset', 'kappaH_onset', 'kappaV_onset',
'arc_length_onset', 'touch_count', 'delta_theta', 'delta_phi',
'delta_kappaH', 'delta_kappaV', 'touch_duration', 'slide_distance']
lick_names = ['num_lick_left', 'num_lick_right']
whisking_names = ['num_whisks', 'midpoint', 'amplitude']
reward_names = ['first_reward_lick_left', 'first_reward_lick_right']
sound_names = ['pole_in_frame', 'pole_out_frame']

In [16]:
theta = merged_df.theta_onset.values
np.nanmean(theta)

-7.774826430871863

In [17]:
np.nanstd(theta)

10.809203189576923

In [18]:
theta_std = (theta - np.nanmean(theta)) / np.nanstd(theta)

In [20]:
nan_inds = np.where(np.isnan(theta_std))[0]
theta_std[nan_inds] = np.nanmean(theta)

In [32]:
for whisker_feature_name in whisker_feature_names:
    if whisker_feature_name != 'touch_count':
        merged_df.loc[:,whisker_feature_name] = standardize_with_nan_values(merged_df.loc[:,whisker_feature_name].values)

merged_df[f'touch_count'] = merged_df[f'touch_count'].apply(lambda x: 0 if np.isnan(x) else x)


In [12]:
whisker_feature_offsets = np.arange(0,3)
whisking_offsets = np.arange(-2,5)
lick_offsets = np.arange(-2,3)
sound_offsets = np.arange(0,5)
reward_offsets = np.arange(0,5)

design_df = merged_df[['trialNum','frame_index']].copy()
for whisker_feature_name in whisker_feature_names:
    for offset in whisker_feature_offsets:
        values_to_assign = merged_df.groupby('trialNum').apply(lambda x: x[whisker_feature_name].shift(offset)).reset_index(drop=True).values
        assert len(values_to_assign) == len(design_df)
        design_df.loc[:,f'{whisker_feature_name}_{offset}'] = values_to_assign
for whisking_name in whisking_names:
    for offset in whisking_offsets:
        values_to_assign = merged_df.groupby('trialNum').apply(lambda x: x[whisking_name].shift(offset)).reset_index(drop=True).values
        assert len(values_to_assign) == len(design_df)
        design_df.loc[:,f'{whisking_name}_{offset}'] = values_to_assign
for lick_name in lick_names:
    for offset in lick_offsets:
        values_to_assign = merged_df.groupby('trialNum').apply(lambda x: x[lick_name].shift(offset)).reset_index(drop=True).values
        assert len(values_to_assign) == len(design_df)
        design_df.loc[:,f'{lick_name}_{offset}'] = values_to_assign
for sound_name in sound_names:
    for offset in sound_offsets:
        values_to_assign = merged_df.groupby('trialNum').apply(lambda x: x[sound_name].shift(offset)).reset_index(drop=True).values
        assert len(values_to_assign) == len(design_df)
        design_df.loc[:,f'{sound_name}_{offset}'] = values_to_assign
for reward_name in reward_names:
    for offset in reward_offsets:
        values_to_assign = merged_df.groupby('trialNum').apply(lambda x: x[reward_name].shift(offset)).reset_index(drop=True).values
        assert len(values_to_assign) == len(design_df)
        design_df.loc[:,f'{reward_name}_{offset}'] = values_to_assign

In [ ]:
# from tqdm.notebook import tqdm
# for i, row in tqdm(use_mice_df.iterrows()):
#     mouse = row['mouse']
#     plane = row['plane']
#     session = int(row['session'])
#     design_df, _ = make_design_dataframe(mouse, plane, session, base_dir)
#     save_dir = base_dir / f'{mouse:03}/plane_{plane}/{session:03}/plane0/roi/glm/whisker_combined'
#     save_dir.mkdir(exist_ok=True, parents=True)
#     design_df.to_pickle(save_dir / 'whisker_combined_design.pkl')

# Run this using scripts\design_matrix_whisker_combined_par.py

# whisker combined GLM

In [15]:
mouse = 25
plane = 1
session = 1

standardize_features=True
standardize_traces=True

roi_dir = base_dir / f'{mouse:03}/plane_{plane}/{session:03}/plane0/roi'
glm_dir = roi_dir / f'glm/whisker_combined'
design_df = pd.read_pickle(glm_dir / f'design_whisker_combined.pkl')

spks = np.load(roi_dir / 'spks_reduced.npy')
iscell = np.load(roi_dir / 'iscell.npy')
cell_inds = np.where(iscell[:,0]==1)[0]
spks = spks[cell_inds,:]
norm_spks = (spks - spks.mean(axis=1)[:,np.newaxis]) / spks.std(axis=1)[:,np.newaxis]
ophys_frametime = pd.read_pickle(roi_dir / 'refined_frame_time.pkl')
assert len(ophys_frametime) == spks.shape[1]

# filter out rows from design_df
# those with NaN values
# those that are not in ophys_frametime (trialNum, frame_index)
keep_ind = np.where(np.isnan(np.sum(design_df.values, axis=1).astype(float))==False)[0]
filtered_design_df = design_df.iloc[keep_ind]
if standardize_features:
    feature_namebase_to_standardize = ['touch_count', 'num_lick', 'num_whisks', 'midpoint', 'amplitude']
        # Other 11 whisker features are already standardized
    feature_names_to_standardize = [design_column for design_column in filtered_design_df.columns if any([namebase in design_column for namebase in feature_namebase_to_standardize])]
    for feature_name in feature_names_to_standardize:
        design_df[feature_name] = (design_df[feature_name] - design_df[feature_name].mean()) / design_df[feature_name].std()
filtered_design_df = filtered_design_df.query('trialNum in @ophys_frametime.trialNum and frame_index in @ophys_frametime.frame_index')
filtered_design_df = filtered_design_df.reset_index(drop=True)
assert len(filtered_design_df.frame_index.unique()) == len(filtered_design_df)
assert np.isin(filtered_design_df.frame_index.values, ophys_frametime.frame_index.values).all()

spks_frame_inds = np.where(np.isin(ophys_frametime.frame_index.values, filtered_design_df.frame_index.values))[0]
assert len(spks_frame_inds) == len(filtered_design_df)
if standardize_traces:
    spks = norm_spks
traces = spks[:,spks_frame_inds].T 
# Now traces are in shape of (n_frames, n_cells)

# Standardization
# No change in touch, lick, reward, sound, num_whisks
# Just for amplitude and midpoint
standardized_names = [key for key in filtered_design_df.keys() if ('midpoint' in key) or ('amplitude' in key)]
for key in standardized_names:
    filtered_design_df[key] = (filtered_design_df[key] - filtered_design_df[key].mean()) / filtered_design_df[key].std()

# feature names
whisker_feature_names_base = ['theta_onset', 'phi_onset', 'kappaH_onset', 'kappaV_onset',
    'arc_length_onset', 'touch_count', 'delta_theta', 'delta_phi',
    'delta_kappaH', 'delta_kappaV', 'touch_duration', 'slide_distance']
whisker_feature_names = [key for key in filtered_design_df.keys() if sum([wfnb in key for wfnb in whisker_feature_names_base])==1]
whisking_names = [key for key in filtered_design_df.keys() if ('num_whisks' in key) or ('midpoint' in key) or ('amplitude' in key)]
lick_names = [key for key in filtered_design_df.keys() if 'num_lick' in key]
sound_names = [key for key in filtered_design_df.keys() if 'pole_in_frame' in key or 'pole_out_frame' in key]
reward_names = [key for key in filtered_design_df.keys() if 'first_reward_lick' in key]

# Adding the bias column
X = np.hstack((np.ones((len(filtered_design_df),1)), filtered_design_df[whisker_feature_names + whisking_names + lick_names + sound_names + reward_names].values)).astype(float)

# Turning into xarray
x = np.hstack((np.ones((len(filtered_design_df),1)), filtered_design_df[whisker_feature_names + whisking_names + lick_names + sound_names + reward_names].values)).astype(float)
X = xr.DataArray(x, dims=('index', 'feature'), 
                    coords={'index':filtered_design_df.index.values,
                            'feature':['intercept'] + whisker_feature_names + whisking_names + lick_names + sound_names + reward_names})
traces = xr.DataArray(traces, dims=('index', 'cell_id'),
                    coords={'index':filtered_design_df.index.values,
                            'cell_id':cell_inds})

In [12]:
key = filtered_design_df.keys()[2]


[True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False]

In [14]:
whisker_feature_names = [key for key in filtered_design_df.keys() if sum([wfnb in key for wfnb in whisker_feature_names_base])==1]
whisker_feature_names


['theta_onset_0',
 'theta_onset_1',
 'theta_onset_2',
 'phi_onset_0',
 'phi_onset_1',
 'phi_onset_2',
 'kappaH_onset_0',
 'kappaH_onset_1',
 'kappaH_onset_2',
 'kappaV_onset_0',
 'kappaV_onset_1',
 'kappaV_onset_2',
 'arc_length_onset_0',
 'arc_length_onset_1',
 'arc_length_onset_2',
 'touch_count_0',
 'touch_count_1',
 'touch_count_2',
 'delta_theta_0',
 'delta_theta_1',
 'delta_theta_2',
 'delta_phi_0',
 'delta_phi_1',
 'delta_phi_2',
 'delta_kappaH_0',
 'delta_kappaH_1',
 'delta_kappaH_2',
 'delta_kappaV_0',
 'delta_kappaV_1',
 'delta_kappaV_2',
 'touch_duration_0',
 'touch_duration_1',
 'touch_duration_2',
 'slide_distance_0',
 'slide_distance_1',
 'slide_distance_2']